In [2]:
pip install keras tensorflow opencv-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from keras.preprocessing import image
import cv2


categories = ['with_mask', 'without_mask']

data = []

for category in categories:
    path = os.path.join('train',category)
    # print(path,os.listdir(path))

    label = categories.index(category)

    for file in os.listdir(path):
        image_path = os.path.join(path,file)
        img = cv2.imread(image_path)
        img = cv2.resize(img,(224,224))

        data.append([img,label])







In [3]:
len(data)

1315

In [4]:

import random

random.shuffle(data)

x=[]
y=[]

for features,label in data:
    x.append(features)
    y.append(label)



In [5]:
import numpy as np

x = np.array(x)
y = np.array(y)


In [6]:
x.shape

(1315, 224, 224, 3)

In [7]:
y.shape

(1315,)

In [8]:
x = x/255

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [10]:
x_train.shape

(1052, 224, 224, 3)

In [11]:
x_test.shape

(263, 224, 224, 3)

In [12]:
from keras.applications.vgg16 import VGG16

In [13]:
vgg = VGG16()

In [14]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
from keras import Sequential

model = Sequential()


for layer in vgg.layers[:-1]:
    model.add(layer)

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [17]:
for layer in model.layers:
    layer.trainable = False

In [18]:
from keras.layers import Dense

model.add(Dense(1,activation="sigmoid"))

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [20]:
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
33/33 [==============================] - 108s 3s/step - loss: 0.6056 - accuracy: 0.6816 - val_loss: 0.4375 - val_accuracy: 0.8707
Epoch 2/5
33/33 [==============================] - 92s 3s/step - loss: 0.3808 - accuracy: 0.8935 - val_loss: 0.3351 - val_accuracy: 0.8973
Epoch 3/5
33/33 [==============================] - 92s 3s/step - loss: 0.2923 - accuracy: 0.9135 - val_loss: 0.2921 - val_accuracy: 0.8897
Epoch 4/5
33/33 [==============================] - 92s 3s/step - loss: 0.2497 - accuracy: 0.9230 - val_loss: 0.2513 - val_accuracy: 0.9240
Epoch 5/5
33/33 [==============================] - 91s 3s/step - loss: 0.2220 - accuracy: 0.9392 - val_loss: 0.2280 - val_accuracy: 0.9125


In [21]:
cap = cv2.VideoCapture(0)


def detect_mask(img):

    y_predi = model.predict(img.reshape(1,224,224,3))

    return y_predi[0][0]


In [ ]:
# def detect_mask(img):

#     try:
#         y_predi = model.predict(img.reshape(1,224,224,3))

#         print(y_predi,img)
#         return y_predi[0][0]
#     except Exception as e:
#         print("error",e)



while True:

    ret, frame = cap.read()

    # detection method Called
    img_resize = cv2.resize(frame,(224,224))

    y_pred = detect_mask(img_resize)

    # image show

    cv2.imshow('window',frame)

    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

cv2.destroyAllWindows()

In [29]:
# from keras.preprocessing.image import img_to_array


def detect_mask(img):

    y_predi = model.predict(img.reshape(1,224,224,3))

    return y_predi[0][0]


sample1 = cv2.imread('./test/without_mask/159.jpg')
sample1 = cv2.resize(sample1,(224,224))
detect_mask(sample1)

1/1 [==============================] - 0s 131ms/step


0.99999845